In [2]:
import pandas as pd
import numpy as np
import os

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
file_path = '{}/nem-data/nemde_clean/df_DUID_CO2_mean.pkl'.format(os.environ['HOME'])
df = pd.read_pickle(file_path)

print(df.index.min())
print(df.index.max())
print(df.shape)
df.head()

2009-07-01 04:00:00
2019-08-01 03:55:00
(1060704, 2)


,Price,CO2E_EMISSIONS_FACTOR
2019-08-01 03:55:00,79.52391,0.380340
2019-08-01 03:50:00,79.17710,0.456409
2019-08-01 03:45:00,85.14416,0.573436
2019-08-01 03:40:00,84.78689,0.573436
2019-08-01 03:35:00,84.78346,0.573436


In [5]:
df.isna().any()

Price                    True
CO2E_EMISSIONS_FACTOR    True
dtype: bool

In [12]:
df = df.iloc[900000:]
print(df.shape)
df.head()

(11171, 3)


,Price,CO2E_EMISSIONS_FACTOR,CO2_group
PeriodID,,,
2018-05-24 01:05:00,107.90144,0.872190,0.828778
2018-05-24 01:10:00,105.00000,0.690287,0.663022
2018-05-24 01:15:00,102.88619,0.000000,-0.000001
2018-05-24 01:20:00,102.06515,0.000000,-0.000001
2018-05-24 01:25:00,89.00000,0.690287,0.663022


In [6]:
def lag_horizon(df, lag, horizon):
    lst = []
    lst_h = []
    
    [lst.append(list(k[i:i+lag])) for j,k in df.loc[:,["CO2E_EMISSIONS_FACTOR"]].iteritems() for i in range(df.shape[0]-lag)]
    [lst.append(np.nan) for i in range(df.shape[0]-lag,df.shape[0])]
    
    [lst_h.append(list(k[i+lag:i+lag+horizon])) for j,k in df.loc[:,["CO2E_EMISSIONS_FACTOR"]].iteritems() for i in range(df.shape[0]-(lag-1)-horizon)]
    [lst_h.append(np.nan) for i in range(df.shape[0]-(lag-1)-horizon,df.shape[0])]
        
    return np.array(lst), np.array(lst_h)

In [14]:
e,f = lag_column(df, lag=3, horizon=3)

In [24]:
print(type(e))
print(type(f))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [15]:
print(len(f))
print(len(e))

11171
11171


In [38]:
df["features"] = e
df["features"] = df["features"].apply(lambda x: np.array(x))

df["target"] = f
df["target"] = df["target"].apply(lambda x: np.array(x))

In [40]:
print(type(df.features.iloc[4]))
print(type(df.target.iloc[4]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [42]:
df.iloc[-10:]

,Price,CO2E_EMISSIONS_FACTOR,CO2_group,features,target
PeriodID,,,,,
2018-07-31 23:10:00,50.88122,0.908305,0.828778,"[0.90830509, 0.91014305, 0.97176136]","[1.039465356, 0.97176136, 0.97176136]"
2018-07-31 23:15:00,47.00597,0.910143,0.828778,"[0.91014305, 0.97176136, 1.039465356]","[0.97176136, 0.97176136, 0.90830509]"
2018-07-31 23:20:00,45.83206,0.971761,0.828778,"[0.97176136, 1.039465356, 0.97176136]","[0.97176136, 0.90830509, 0.91014305]"
2018-07-31 23:25:00,52.82719,1.039465,0.994534,"[1.039465356, 0.97176136, 0.97176136]","[0.90830509, 0.91014305, 0.7281144399999999]"
2018-07-31 23:30:00,72.13931,0.971761,0.828778,"[0.97176136, 0.97176136, 0.90830509]","[0.91014305, 0.7281144399999999, 0.91014305]"
2018-07-31 23:35:00,76.44245,0.971761,0.828778,"[0.97176136, 0.90830509, 0.91014305]",NaN
2018-07-31 23:40:00,74.04497,0.908305,0.828778,"[0.90830509, 0.91014305, 0.7281144399999999]",NaN
2018-07-31 23:45:00,69.94850,0.910143,0.828778,NaN,NaN
2018-07-31 23:50:00,69.24109,0.728114,0.663022,NaN,NaN


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11171 entries, 2018-05-24 01:05:00 to 2018-07-31 23:55:00
Data columns (total 5 columns):
Price                    11171 non-null float64
CO2E_EMISSIONS_FACTOR    11171 non-null float64
CO2_group                11171 non-null category
features                 11168 non-null object
target                   11166 non-null object
dtypes: category(1), float64(2), object(2)
memory usage: 447.7+ KB


In [45]:
df.features.iloc[4]

array([0.69028667, 0.        , 0.69028667])

In [44]:
df.features.iloc[4] + df.features.iloc[4]

array([1.38057333, 0.        , 1.38057333])

In [22]:
lst = [1,2,3]
type(np.array(lst))

numpy.ndarray